In [ ]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import os
import json
from urllib.parse import parse_qs

from pycountry import countries

#You should not change this lines.
#They define the parameters required to run the server.
hostName = "localhost"
serverPort = 8080


#MILESTONE 1
#endpoint => http://localhost:8080/test_connectivity_M1

#You just need to test the connectivity that returns the following JSON
#{"results": [{"Connectivity": "OK"}]}

#JSON TO DISPLAY IN CLIENT:
#{"results": [{"Connectivity": "OK"}]}

def test_connectivity_M1(text):
    results = {}
    json_result={}
    array = []             
    json_result['Connectivity']="OK"
    array.append(json_result)
    results['results'] = array
    api_answer=json.dumps(results, ensure_ascii=False)
    output_code=200
    return api_answer,output_code

#MILESTONE 2
#endpoint => http://localhost:8080/count_countries_M2
#Country Count. Provide the number of entries per country 
#in the dataset in a sorted way. First the most frequent country.
#TEXT TO DISPLAY IN CLIENT:
# 1: TR(317)
# 2: DE(317)
# 3: NO(315)
# 4: BR(312)
# 5: IR(311)
# 6: CA(297)
# 7: FR(297)
# 8: CH(296)
# 9: IE(293)
# 10: DK(290)
# 11: NZ(288)
# 12: AU(285)
# 13: ES(281)
# 14: GB(278)
# 15: NL(276)
# 16: FI(276)
# 17: US(271)

with open('json_rand_users_5000.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

def country_count_M2(text):
    results = {}
    json_result={}
    array = [] 

    for user in data['results']:
        country = user.get('nat', None)
        if country:
            json_result[country] = json_result.get(country, 0) + 1
    
    json_result = sorted(json_result.items(), key=lambda x: x[1], reverse=True)
    rank = 1
    for country, count in json_result:
        array.append(f"{rank}: {country}({count})")
        rank += 1
    results['results'] = array
    api_answer=json.dumps(results, ensure_ascii=False)
    output_code=200
    return api_answer,output_code
    
#MILESTONE 3    
#endpoint=>http://localhost:8080/count_countries_M3?countries=DK,ES,GB,IE,IR,NO,NL,NZ

#It is the same than before but in case it just returns the result for the provided countries.
#The list of available countries in the raduser API (field nat) is:
#AU, BR, CA, CH, DE, DK, ES, FI, FR, GB, IE, IR, NO, NL, NZ, TR, US
#If not countries parameter is provided you have to provide the results for all the countries
#NOTE: NO NEED TO IMPLEMENT CONTROL OF ERRORS

#TEXT TO DISPLAY IN CLIENT THAT ONLY CONSIDERS THE COUNTRIES IN THE QUERY:
# 1: NO(315)
# 2: IR(311)
# 3: IE(293)
# 4: DK(290)
# 5: NZ(288)
# 6: ES(281)
# 7: GB(278)
# 8: NL(276)

def country_count_M3(text):   
    
    results = {}
    json_result={}
    array = []       

    query = parse_qs(text.split('?')[1] if '?' in text else {}) 
    countries = query.get('countries', None)
    if countries:
        countries = countries[0].split(',')
    
    available_countries = ['AU', 'BR', 'CA', 'CH', 'DE', 'DK', 'ES', 'FI', 'FR', 'GB', 'IE', 'IR', 'NO', 'NL', 'NZ', 'TR', 'US']

    if countries:
        countries = [country for country in countries if country in available_countries]
    else:
        countries = available_countries

    for user in data['results']:
        country = user.get('nat', None)
        if country and country in countries:
            json_result[country] = json_result.get(country, 0) + 1
    
    json_result = sorted(json_result.items(), key=lambda x: x[1], reverse=True)
    rank = 1
    for country, count in json_result:
        array.append(f"{rank}: {country}({count})")
        rank += 1

    results['results'] = array
    api_answer=json.dumps(results, ensure_ascii=False)
    output_code=200
    return api_answer,output_code

#MILESTONE 4    
#endpoint=>http://localhost:8080/age_groups_M4 
#You need to deliver the number of users in the data set
#within the following age groups: <18, 18-29, 30-49, 50-64, >64.

#ANSWER IN CLIENT
# Age <18: 0
# Age 18-29: 634
# Age 30-49: 1762
# Age 50-64: 1452
# Age >64: 1152

def age_groups_M4(text):
  
    results = {}
    json_result={}
    array = []      
    
    age_groups = {
        'Age <18': 0,
        'Age 18-29': 0,
        'Age 30-49': 0,
        'Age 50-64': 0,
        'Age >64': 0
    }
    
    for user in data['results']:
        age = user.get('dob', {}).get('age', None)
        if age:
            if age < 18:
                age_groups['Age <18'] += 1
            elif age < 30:
                age_groups['Age 18-29'] += 1
            elif age < 50:
                age_groups['Age 30-49'] += 1
            elif age < 65:
                age_groups['Age 50-64'] += 1
            else:
                age_groups['Age >64'] += 1
    
    array.append(f"Age <18: {age_groups['Age <18']}")
    array.append(f"Age 18-29: {age_groups['Age 18-29']}")
    array.append(f"Age 30-49: {age_groups['Age 30-49']}")
    array.append(f"Age 50-64: {age_groups['Age 50-64']}")
    array.append(f"Age >64: {age_groups['Age >64']}")
    
    results['results'] = array
    api_answer=json.dumps(results, ensure_ascii=False)
    output_code=200
    return api_answer,output_code

#MILESTONE 5
#endpoint => http://localhost:8080/users_M5?countries=FR,GB,IE,IR,NO,NL,NZ,TR,US&gender=female&month=01,02,03,04,05&age=30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45
#You need to find users meeting the list of parameters in the query.
#The query should work with 1, 2, 3 or 4 of the fields combined in whatever

#countries=AU,BR,CA,CH,DE,DK,ES,FI,FR,GB,IE,IR,NO,NL,NZ,TR,US  [use the nat field for this] (if not defined consider all countries)
#gender=male,female (if not used consider all genders)
#month= 01,02,03,04,05,06,07,08,09,10,11,12 (if not used consider all the months)
#age:Any value between 18 and 99 (if not used consider all ages)

#NOTE1: IT IS NOT NEEDED TO DO CONTROL ERRORS. 
#NOTE2: You need to provide the total number of users and the 100 first users in the JOSN as follows:
#NOTE3: some times the query selection my obtain less than 100 users then print all of them

#TEXT TO DISPLAY IN CLIENT:

# NUM. USERS:165
# User 1:
# 	-Name: Linne Hoogkamer
# 	-Gender: female
# 	-Age: 40
# 	-Country: Netherlands
# 	-Postcode: 42173
# 	-Email: linne.hoogkamer@example.com
# User 2:
# 	-Name: Enola Martin
# 	-Gender: female
# 	-Age: 43
# 	-Country: France
# 	-Postcode: 93894
# 	-Email: enola.martin@example.com
# User 3:
# 	-Name: Mathilda Skjevik
# 	-Gender: female
# 	-Age: 42
# 	-Country: Norway
# 	-Postcode: 6630
# 	-Email: mathilda.skjevik@example.com
# User 4:
# 	-Name: Valentine Thomas
# 	-Gender: female
# 	-Age: 35
# 	-Country: France
# 	-Postcode: 84452
# 	-Email: valentine.thomas@example.com
#.....
#.....
#.....
#User 100:
# 	-Name: Cecilia Asphaug
# 	-Gender: female
# 	-Age: 42
# 	-Country: Norway
# 	-Postcode: 5052
# 	-Email: cecilia.asphaug@example.com

def users_M5(text):
    
    query = parse_qs(text.split('?')[1] if '?' in text else {})
    
    countries = query.get('countries', [None])
    if countries:
        countries = countries[0].split(',')
    
    genders = query.get('gender', [None])[0]
    if genders:
        genders = genders.split(',')
    
    months = query.get('month', [None])[0]
    if months:
        months = months.split(',')
        
    ages = query.get('age', [None])[0]
    if ages:
        ages = list(map(int, ages.split(',')))
    
    users = []
    
    for user in data['results']:
        if countries and user.get('nat') not in countries:
            continue
            
        if genders and user.get('gender') not in genders:
            continue
        
        birth_month = user.get('dob', {}).get('date', '')[5:7]
        if months and birth_month not in months:    
            continue
        
        age = user.get('dob', {}).get('age', None)
        if ages and age not in ages:
            continue
        
        users.append({
            'name': f"{user['name']['first']} {user['name']['last']}",
            'gender': user['gender'],
            'age': age,
            'country': user['location']['country'],
            'postcode': user['location']['postcode'],
            'email': user['email']
        })
    
    limited_users = users[:100]
    
    results = {}
    json_result={}
    array = []   
    
    num_users = len(users)
    array.append(f"NUM. USERS:{num_users}")
    
    for idx, user in enumerate(limited_users, start=1):
        array.append(f"User {idx}:")
        array.append(f"-Name: {user['name']}")
        array.append(f"-Gender: {user['gender']}")
        array.append(f"-Age: {user['age']}")
        array.append(f"-Country: {user['country']}")
        array.append(f"-Postcode: {user['postcode']}")
        array.append(f"-Email: {user['email']}")
    
    results['results'] = array
    api_answer=json.dumps(results, ensure_ascii=False)
    output_code=200
    return api_answer,output_code

#MILESTONE 6
#We have to repeat the same exercise than in M5 but this time we need to implement control error
#FORMAT CONTROL:
# Countries: If a country is not in the available list. Generate an error and inform the user which country(s) is(are) wrong with a mesasge
# Gender: If a gender is not in the list of genders. Generate an error and inform the user which gender(s) is(are) wrong with a message
# Month: If the month is not a valid value (e.g., 15). Generate an error and inform the user which month(s) is(are) wrong with a message
# Age: if the age is not in the range 18-99 it is a wrong value. Generate an error and inform the user which age(s) is(are) wrong with a message
# Field: If after ? there is an field (e.g., extra_field) different that countries, gender, age and month report the error.
# The output message should be json file with an error per category including all the wrong values employed.
#
#NOTES:
#1- The succeed of the process should report a status code 200.
#2- The failure of the process due to an error should report the status 400 (Bad request)
#3- You need to include all the errors so you should not stop the verification of the format when you find the first error.

#TEXT TO DISPLAY IN CLIENT (it is assuming all errors are present in the created json based on the following request: )
 
#JOSN output
#{"results": [{"Country Error": "The following country IDs are not supported: IT,SW,MX", 
#"Gender Error": "The following gender options are not supported: women,men", 
#"Month Error": "The following month options are not supported: 15,23", 
#"Age Error": "The following month options are not supported: 12,15,123", 
#"Field Error": "The following fields are not valid: city"}]}

#IF THERE IS AN ERROR ONLY IN 1 or 2 FIELDS THE JSON SHOULD ONLY INCLUDE THOSE ERRORS

valid_countries = ['AU', 'BR', 'CA', 'CH', 'DE', 'DK', 'ES', 'FI', 'FR', 'GB', 'IE', 'IR', 'NO', 'NL', 'NZ', 'TR', 'US']
valid_genders = ['male', 'female']
valid_months = [f"{i:02d}" for i in range(1, 13)]  # '01', '02', ..., '12'
valid_fields = ['countries', 'gender', 'age', 'month']

def users_control_error_M6(text):
    
    query = parse_qs(text.split('?')[1] if '?' in text else {})
    
    country_errors = []
    gender_errors = []
    month_errors = []
    age_errors = []
    field_errors = []
    
    for key in query.keys():
        if key not in valid_fields:
            field_errors.append(key)
    
    countries = query.get('countries', [None])[0]
    if countries:
        countries = countries.split(',')
        invalid_countries = [c for c in countries if c not in valid_countries]
        if invalid_countries:
            country_errors.append(f"The following country IDs are not supported: {', '.join(invalid_countries)}")
    
    
    genders = query.get('gender', [None])[0]
    if genders:
        genders = genders.split(',')
        invalid_genders = [g for g in genders if g not in valid_genders]
        if invalid_genders:
            gender_errors.append(f"The following gender options are not supported: {gender}")

    months = query.get('month', [None])[0]
    if months:
        months = months.split(',')
        invalid_months = [m for m in months if m not in valid_months]
        if invalid_months:
            month_errors.append(f"The following month options are not supported: {', '.join(invalid_months)}")
            
    errors = {}
    if country_errors:
        errors['Country Error'] = " ".join(country_errors)
    if gender_errors:
        errors['Gender Error'] = " ".join(gender_errors)
    if month_errors:
        errors['Month Error'] = " ".join(month_errors)
    if age_errors:
        errors['Age Error'] = " ".join(age_errors)
    if field_errors:
        errors['Field Error'] = f"The following fields are not valid: {', '.join(field_errors)}"
    
    if errors:
        results = {"results": [errors]}
        api_answer = json.dumps(results, ensure_ascii=False)
        output_code = 400
        return api_answer, output_code
    ages = query.get('age', [None])[0]
    if ages:
        ages = list(map(int, ages.split(',')))
        invalid_ages = [a for a in ages if a < 18 or a > 99]
        if invalid_ages:
            age_errors.append(f"The following age options are not supported: {', '.join(map(str, invalid_ages))}")
    
    errors = {}
    if country_errors:
        errors['Country Error'] = " ".join(country_errors)
    if gender_errors:
        errors['Gender Error'] = " ".join(gender_errors)
    if month_errors:
        errors['Month Error'] = " ".join(month_errors)
    if age_errors:
        errors['Age Error'] = " ".join(age_errors)
    if field_errors:
        errors['Field Error'] = f"The following fields are not valid: {', '.join(field_errors)}"
    
    # If there are errors, return the error response
    if errors:
        results = {"results": [errors]}
        api_answer = json.dumps(results, ensure_ascii=False)
        output_code = 400
        return api_answer, output_code
    else: 
        print(text)
        new_text = text.replace('users_control_error_M6', 'users_M5')
        return users_M5(new_text)

#MILESTONE 7
#endpoint => http://localhost:8080/access_token_M7?access_token=$P$BZlw7PF1j.XDezr9sUj6moCjBlSx/e0&countries=FR,GB,IE,IR,NO,NL,NZ,TR,US
#&user_ID=9123&gender=male,female&month=01,05,06,12&age=30,31,32,33,34,35

#We have to repeat the same exercise than in M5 but this time we need to controll whether the userID and access token are correct.
#To this end we will use a query as follow where the access token parameter appears as another field. But it is a compulsory field.
#So we basically need to verify if the access token is there and if so we call the M6 endpoint.
#The URL will look like as follow:
#http://localhost:8080/access_token_M7?userID=127821&access_token=XXASJasnsna?.12123jk2amsass?&countries=FR,ES&age=18,19,20,21,22&gender=male
#You need to validate the user ID and access token in the list of valids access tokens.


#TEXT OTPIONS TO DISPLAY IN CLIENT FOR ERRORS (if access token and user ID correct then deliver answer associated to M6)

#1- No fields included (could be one of them or both of them) 
#{"results": [{"user_ID Error": "user_ID field not included in the parameters.", 
#"Access_Token Error": "Access_Token field not included in the parameters."}]}

#2- UserID does not exist
#{"results": [{"user_ID Error": "user_ID 2131 not found in the database."}]}

#3- User ID exists but access_token wrong
#{"results": [{"access_token Error": "access_token 7423hjmn23nsd.121 wrong for the userID 9123"}]}

import csv
import re

user_access_tokens = {}
with open('file_access_token_M7.csv', 'r') as f:
    reader = csv.reader(f, delimiter=';')
    next(reader)  # Skip the header
    for rows in reader:
        user_access_tokens[rows[0]] = rows[1] 
    

def access_token_M7(text):
    
    query = parse_qs(text.split('?')[1]) if '?' in text else {}

    # Check if user_ID and access_token are provided
    user_id = query.get('user_ID', [None])[0]
    access_token = query.get('access_token', [None])[0]
    
    errors = {}
    
    # Error if either user_ID or access_token is missing
    if not user_id:
        errors['user_ID Error'] = "user_ID field not included in the parameters."
    if not access_token:
        errors['Access_Token Error'] = "Access_Token field not included in the parameters."
    
    # If both are missing, return an error
    if errors:
        results = {"results": [errors]}
        api_answer = json.dumps(results, ensure_ascii=False)
        output_code = 400
        return api_answer, output_code
    
    # Check if the user_ID exists in the access token file
    if user_id not in user_access_tokens:
        results = {"results": [{"user_ID Error": f"user_ID {user_id} not found in the database."}]}
        api_answer = json.dumps(results, ensure_ascii=False)
        output_code = 400
        return api_answer, output_code

    # Check if the access_token matches the user_ID
    if user_access_tokens[user_id] != access_token:
        results = {"results": [{"access_token Error": f"access_token {access_token} wrong for the userID {user_id}"}]}
        api_answer = json.dumps(results, ensure_ascii=False)
        output_code = 400
        return api_answer, output_code
    else: 
        new_text = re.sub(r'access_token=[^&]+&|user_ID=[^&]+&', '', text.replace('access_token_M7', 'users_control_error_M6'))
        return users_control_error_M6(new_text)


class MyServer(BaseHTTPRequestHandler):
    def do_GET(self):
        end_point=self.path
        
        if(end_point.startswith("/test_connectivity_M1")):
            api_answer,output_code= test_connectivity_M1(end_point)
            
        elif(end_point.startswith("/count_countries_M2")):
            api_answer,output_code= country_count_M2(end_point)

        
        elif(end_point.startswith("/count_countries_M3")):
            api_answer,output_code= country_count_M3(end_point)

            
        elif(end_point.startswith("/age_groups_M4")):
            api_answer,output_code= age_groups_M4(end_point)

            
        elif(end_point.startswith("/users_M5")):
            api_answer,output_code= users_M5(end_point)

        elif(end_point.startswith("/users_control_error_M6")):
            api_answer,output_code= users_control_error_M6(end_point)
        
        elif(end_point.startswith("/access_token_M7")):
            api_answer,output_code= access_token_M7(end_point)

        
        else:
            results = {}
            json_error={}
            array = []             
            json_error['Endpoint Error']="The provided endpoint does not exist."
            array.append(json_error)
            results['results'] = array
            api_answer=json.dumps(results, ensure_ascii=False)
            output_code=400
        
        self.send_response(output_code)
        self.send_header("Content-type", "text/json")
        self.end_headers() 
        self.wfile.write(bytes(api_answer, "utf-8"))
        
if __name__ == "__main__":        
    webServer = HTTPServer((hostName, serverPort), MyServer)
    print("Server started http://%s:%s" % (hostName, serverPort))

    try:
        webServer.serve_forever()
    except KeyboardInterrupt:
        pass

    webServer.server_close()
    print("Server stopped.")



Server started http://localhost:8080
